# **Saisonnalité et durée de la saisonnalité**

## Configuration

In [ ]:
# Set SNT Paths
ROOT_PATH  <- "~/workspace"
CODE_PATH      <- file.path(ROOT_PATH, "code")
CONFIG_PATH    <- file.path(ROOT_PATH, "configuration")
DATA_PATH <- file.path(ROOT_PATH, 'data')
OUTPUT_DATA_PATH <- file.path(DATA_PATH, 'seasonality_cases')
PLOTS_PATH <- file.path(ROOT_PATH, "pipelines/snt_seasonality_cases/reporting/outputs")

# load util functions
source(file.path(CODE_PATH, "snt_utils.r"))

# List required packages 
required_packages <- c(
    "glue",
    # "terra",
    "ggplot2",
    "stringr",
    "lubridate",
    # "viridis",
    "arrow",
    "sf",
    "reticulate",
    "data.table",
    "RColorBrewer"
)

# Execute function
install_and_load(required_packages)

# Set environment to load openhexa.sdk from the right environment
Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")
reticulate::py_config()$python
openhexa <- import("openhexa.sdk")

# Load SNT config
config_json <- tryCatch({ jsonlite::fromJSON(file.path(CONFIG_PATH, "SNT_config.json"))},
    error = function(e) {
        msg <- paste0("Error while loading configuration", conditionMessage(e))  
        cat(msg)   
        stop(msg) 
    })

# Required environment for the sf packages
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.setenv(GDAL_DATA = "/opt/conda/share/gdal")

# Configuration variables
COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE
dhis2_dataset <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED

In [ ]:
# Parameters
minimum_month_block_size <- as.integer(3)
maximum_month_block_size <- as.integer(5)
threshold_for_seasonality <- 0.6
threshold_proportion_seasonal_years <- 0.5

In [ ]:
# Global variables
type_of_seasonality <- "cases"
formatted_threshold_for_seasonality <- sprintf("%d%%", round(threshold_for_seasonality * 100))
data_source <- "DHIS2"

# space and time columns
admin_level <- 'ADM2'
admin_id_col <- paste(admin_level, toupper('id'), sep = '_')
admin_name_col <- paste(admin_level, toupper('name'), sep = '_')
year_col <- 'YEAR'
month_col <- 'MONTH'
period_cols <- c(year_col, month_col)

## Saisonnalité estimée sur la base des données pluviométriques

In [ ]:
# Load spatial file from dataset
spatial_data_filename <- paste(COUNTRY_CODE, "shapes.geojson", sep = "_")

spatial_data <- tryCatch({ get_latest_dataset_file_in_memory(dhis2_dataset, spatial_data_filename) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 Shapes data for: " , COUNTRY_CODE, conditionMessage(e))
                      cat(msg)
                      stop(msg)
                      })

In [ ]:
# import seasonality data
seasonality_data <- read_parquet(file.path(OUTPUT_DATA_PATH, glue("{COUNTRY_CODE}_cases_seasonality.parquet")))
setDT(seasonality_data)

In [ ]:
# merge polygon file with seasonality dataset
plot_data <- merge(spatial_data, seasonality_data, by = c('ADM1_NAME','ADM1_ID', admin_id_col, admin_name_col), all = TRUE)

## Create plots

In [ ]:
# the "existence of seasonality" plot
seasonality_plot <- make_seasonality_plot(
  spatial_seasonality_df=plot_data,
  seasonality_colname=paste('SEASONALITY', toupper(type_of_seasonality), sep = "_"),
  title_label=paste("Seasonality:", formatted_threshold_for_seasonality, type_of_seasonality, sep = ' ')
)

filename_seasonality_plot <- paste(COUNTRY_CODE, data_source, admin_level, gsub("\\.", "", as.character(threshold_for_seasonality)), type_of_seasonality, 'seasonality_plot.png', sep = '_')
ggsave(seasonality_plot, file = file.path(PLOTS_PATH, filename_seasonality_plot), dpi = 500)

## Beginning month of seasonal block

In [ ]:
# the "beginning month of seasonality" plot
season_start_month_col <- paste('SEASONAL_BLOCK_START_MONTH', toupper(type_of_seasonality), sep = "_")

if (season_start_month_col %in% names(plot_data)) {
  
  # Create month labels
  month_labels <- c("1" = "January", "2" = "February", "3" = "March", "4" = "April",
                     "5" = "May", "6" = "June", "7" = "July", "8" = "August",
                     "9" = "September", "10" = "October", "11" = "November", "12" = "December")
  
  # Convert to factor with month labels
  plot_data$START_MONTH_FACTOR <- factor(
    as.character(plot_data[[season_start_month_col]]),
    levels = as.character(1:12),
    labels = month_labels
  )
  
  # Color palette for 12 months (using distinct, visible colors)
  month_colors <- c(
    "January" = "#9E0142",    # Dark red/purple
    "February" = "#D53E4F",    # Red
    "March" = "#F46D43",       # Orange-red
    "April" = "#FDAE61",      # Orange
    "May" = "#FEE08B",        # Yellow
    "June" = "#E6F598",       # Light yellow-green
    "July" = "#ABDDA4",    # Light green
    "August" = "#66C2A5",       # Teal-green
    "September" = "#3288BD", # Blue
    "October" = "#5E4FA2",    # Purple-blue
    "November" = "#C51B7D",   # Magenta
    "December" = "#8E0152"    # Dark purple
  )
  
  # Create the beginning month map
  start_month_plot <- ggplot(plot_data) +
    geom_sf(aes(fill = START_MONTH_FACTOR), color = 'black', size = 0.1) +
    scale_fill_manual(
      values = month_colors,
      name = 'Beginning month',
      na.value = 'grey90',
      drop = FALSE,
      guide = guide_legend(nrow = 2)
    ) +
    coord_sf() +
    theme_classic() +
    theme(
      plot.title = element_text(face = "bold", hjust = 0.5, size = 12),
      legend.position = "bottom",
      legend.key.width = unit(1.5, "cm"),
      legend.text = element_text(size = 9)
    ) +
    labs(title = paste("Beginning month of seasonal block -", COUNTRY_CODE))
  
  print(start_month_plot)
  
  # Save the plot
  filename_start_month_plot <- paste(COUNTRY_CODE, data_source, admin_level, gsub("\\.", "", as.character(threshold_for_seasonality)), type_of_seasonality, 'start_month_plot.png', sep = '_')
  ggsave(start_month_plot, file = file.path(PLOTS_PATH, filename_start_month_plot), dpi = 500, width = 12, height = 8)
  
} else {
  cat(paste('Column', season_start_month_col, 'not found in data. Run the updated code notebook first.\n'))
}

In [ ]:
names(plot_data)

In [ ]:
# the "duration of seasonality" plot
duration_plot <- make_seasonality_duration_plot(
  spatial_seasonality_df=plot_data,
  seasonality_duration_colname=paste('SEASONAL_BLOCK_DURATION', toupper(type_of_seasonality), sep = "_"),
  title_label=paste("Months:", formatted_threshold_for_seasonality, type_of_seasonality, sep=' '),
  palette_name="Spectral",
  none_label="Not seasonal"
)


filename_duration_plot <- paste(COUNTRY_CODE, data_source, admin_level, gsub("\\.", "", as.character(threshold_for_seasonality)), type_of_seasonality, 'duration_plot.png', sep = '_')

ggsave(duration_plot, file = file.path(PLOTS_PATH, filename_duration_plot), dpi = 500)

## Proportion de cas dans le bloc saisonnier

In [ ]:
# Map: CASES_PROPORTION (proportion of annual cases in seasonal block)
if ('CASES_PROPORTION' %in% names(plot_data)) {
  
  # Create proportion categories
  plot_data$PROPORTION_CAT <- cut(
    plot_data$CASES_PROPORTION,
    breaks = c(-Inf, 0, 0.2, 0.4, 0.6, 0.8, 1.0, Inf),
    labels = c('< 0%', '0 - 20%', '20 - 40%', '40 - 60%', '60 - 80%', '80 - 100%', '> 100%'),
    include.lowest = TRUE
  )
  
  # Color palette similar to rainfall
  proportion_palette <- c(
    '< 0%' = '#455A64',
    '0 - 20%' = '#43A047',
    '20 - 40%' = '#8BC34A',
    '40 - 60%' = '#FDD835',
    '60 - 80%' = '#FF9800',
    '80 - 100%' = '#E65100',
    '> 100%' = '#B71C1C'
  )
  
  # Create the proportion map
  proportion_plot <- ggplot(plot_data) +
    geom_sf(aes(fill = PROPORTION_CAT), color = 'black', size = 0.1) +
    scale_fill_manual(
      values = proportion_palette,
      name = 'Proportion',
      na.value = 'grey90',
      drop = FALSE
    ) +
    theme_void() +
    theme(
      legend.position = 'bottom',
      plot.title = element_text(hjust = 0.5, size = 12, face = 'bold')
    ) +
    labs(title = paste('Saisonnalité - Proportion de cas dans le bloc -', COUNTRY_CODE))
  
  print(proportion_plot)
  
  # Save the plot
  filename_proportion_plot <- paste(COUNTRY_CODE, data_source, admin_level, 'cases_proportion_plot.png', sep = '_')
  ggsave(proportion_plot, file = file.path(PLOTS_PATH, filename_proportion_plot), dpi = 500)
  
} else {
  cat('CASES_PROPORTION column not found in data. Run the updated code notebook first.\n')
}